In [1]:
import numpy as np
import pandas as pd

## Задача

Предобработать текст следующим способом:

- разбить текст на слова
- привести все к нижнему регистру
- убрать из текста все точки, запятые и скобки

Найти в тексте самое частовстречаемое слово.

Способ через цикл

In [2]:
import string

def preprocess_text(x):
    res = []
    for word in x.lower().split():
        for sign in string.punctuation:
            word = word.replace(sign, '')
            # print(word)
        res.append(word)
    return res
            

Способ через регулярки (на практике лучше использовать что-то такое)

In [3]:
import re
def preprocess_text(text):
    return re.sub(r"[^\w\s]+", '', text).lower().split()

In [4]:
from collections import Counter

In [5]:
text = "Разобьем этот текст на слова, приведем к нижнему регистру. Затем уберем пунктуацию (точки, запятые и скобки). А потом найдем слово, которое встречается чаще всего. текст текст текст."

# Ваш код здесь
Counter(preprocess_text(text))

Counter({'текст': 4,
         'разобьем': 1,
         'этот': 1,
         'на': 1,
         'слова': 1,
         'приведем': 1,
         'к': 1,
         'нижнему': 1,
         'регистру': 1,
         'затем': 1,
         'уберем': 1,
         'пунктуацию': 1,
         'точки': 1,
         'запятые': 1,
         'и': 1,
         'скобки': 1,
         'а': 1,
         'потом': 1,
         'найдем': 1,
         'слово': 1,
         'которое': 1,
         'встречается': 1,
         'чаще': 1,
         'всего': 1})

## Задача

Написать функцию для обработки предложения при помощи стемминга для английского языка

In [6]:
from nltk.stem.porter import *

stemmer = PorterStemmer()

In [7]:
def preprocess_sentence_eng(text):
    return ' '.join(map(stemmer.stem, preprocess_text(text)))

In [8]:
example = """Unit tests for the Porter stemmer
>>> from nltk.stem.porter import *
Create a new Porter stemmer.
"""

preprocess_sentence_eng(example)

'unit test for the porter stemmer from nltkstemport import creat a new porter stemmer'

## Задача

Применить стемминг к тексту и добавить колонку `text_stemmed`

In [9]:
import pandas as pd

In [10]:
df = pd.read_csv('../data/spam.csv')
df

,text,target
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham
...,...,...
5567,This is the 2nd time we have tried 2 contact u...,spam
5568,Will ? b going to esplanade fr home?,ham
5569,"Pity, * was in mood for that. So...any other s...",ham
5570,The guy did some bitching but I acted like i'd...,ham


In [11]:
df['text_stemmed'] = df['text'].apply(preprocess_sentence_eng)

In [12]:
df['text_stemmed'].head(3)

0    go until jurong point crazi avail onli in bugi...
1                                ok lar joke wif u oni
2    free entri in 2 a wkli comp to win fa cup fina...
Name: text_stemmed, dtype: object

## Задача

Выбрать метрику и модель, векторизовать текст и научиться искать спам

Можно взять любую метрику, которая учитывает дизбаланс классов. Например, `f1_score`

In [13]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [14]:
df['target_num'] = df['target'].map({'ham': 0, 'spam': 1}) # переводим таргет в числа
train, test = train_test_split(df, random_state=0)

In [15]:
bow = CountVectorizer()
x_train = bow.fit_transform(train['text_stemmed'])
x_test = bow.transform(test['text_stemmed'])
y_train = train['target_num']
y_test = test['target_num']

model = LogisticRegression()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
f1_score(y_pred, y_test)

0.9125683060109289

Скопировали - вставили - поменяли на TF-IDF

In [16]:
bow = TfidfVectorizer()
x_train = bow.fit_transform(train['text_stemmed'])
x_test = bow.transform(test['text_stemmed'])
y_train = train['target_num']
y_test = test['target_num']

model = LogisticRegression()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
f1_score(y_pred, y_test)

0.861271676300578

Получилось хуже.

## Задача

Попробовать случайный лес для классификации документов. Использовать пайплайны для решения.

Выборку необходимо как и раньше делить на треин и тест.

Напишем кусочки пайплайна и запустим перебор пайплайнов

In [17]:
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator

In [18]:
class BasePreprocessor(BaseEstimator):
    """
    Класс для базовой обработки текста - нижний регистр и пунктуация
    """
    def fit(self, x, y=None):
        return self
    
    def transform(self, x):
        return np.array(list(map(lambda x: ' '.join(preprocess_text(x)), x)))

In [19]:
BasePreprocessor().transform(['test123test _ tEst - test @ Test'])

array(['test123test _ test test test'], dtype='<U28')

In [20]:
class StemmerEng(BaseEstimator):
    """
    Класс для стемминга на английском языке - разбиваем по словам и применяем к каждому стеммер
    """
    def __init__(self):
        self.stemmer = PorterStemmer()
        
    def fit(self, x, y=None):
        return self
    
    def _stem(self, word):
        return self.stemmer.stem(word)
    
    def _transform_text(self, text):
        return ' '.join(map(self._stem, text.split())) 

    def transform(self, x):
        return list(map(self._transform_text, x))

In [21]:
StemmerEng().transform(['always test tests'])

['alway test test']

In [22]:
fit_results = []

# На вход подаем необработанный текст
x_train = train['text']
x_test = test['text']
y_train = train['target_num']
y_test = test['target_num']

# Перебираем модели и векторизацию
for vect in [CountVectorizer(), TfidfVectorizer()]:
    for model in [LogisticRegression(), RandomForestClassifier()]:
        pipeline = Pipeline(
            [
                ("base", BasePreprocessor()), # препроцессинг тоже можно подбирать
                ("stem", StemmerEng()), # как и стемминг/лемматизацию
                ("vect", vect),
                ("model", model),
            ]
        )
        pipeline.fit(x_train, y_train)
        y_pred = pipeline.predict(x_test)
        metric = f1_score(y_pred, y_test)
        fit_results.append(
            {
                'vect': vect.__class__.__name__,
                'model': model.__class__.__name__,
                'f1': metric,
            }
        )

fit_results = pd.DataFrame(fit_results)

In [23]:
fit_results.sort_values('f1', ascending=False)

,vect,model,f1
0,CountVectorizer,LogisticRegression,0.912568
1,CountVectorizer,RandomForestClassifier,0.874286
3,TfidfVectorizer,RandomForestClassifier,0.874286
2,TfidfVectorizer,LogisticRegression,0.861272


## Задача

Используя word2vec классифицировать тексты

Алгоритм:

- разделить на треин и тест выборки
- обучить word2vec на треин выборке
- написать функцию для составления вектора по документу (для этого нужно посчитать среднее векторов всех слов)
- обучить модель, оценить результаты

Для начала лучше взять небольшую выборку, чтобы проще было писать код (например, 2000 в треин и 5 примеров в тест).

Когда убедились, что все работает, брать весь датасет.

Функции, которые могут помочь и подсказки:

- `model.wv.key_to_index` - словарь токен->номер токена. Можно взять список всех слов отсюда.
- `model.wv[word]` - получаем вектор по слову. Не забываем применить предобработку!
- может случиться так, что word2vec не будет знать какого-то слова из обучающей выборки. Тогда этому слову присваиваем нулевой вектор.

In [24]:
# TBD =)